In [1]:
import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd
import json
from urllib.request import urlopen
from datetime import datetime, timedelta
from random import randrange

In [2]:
# Getting county codes for choropleth map
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [3]:
# Creating a dataframe to reference cities and their asscoiated county
cols = ['city', 'county_name', 'state_id', 'state_name', 'county_fips']
cities_df = pd.read_csv('uscities.csv', usecols=cols)

print(cities_df.columns)

Index(['city', 'state_id', 'state_name', 'county_fips', 'county_name'], dtype='object')


In [4]:
# Creating a dataframe from covid-19 data provided via url
url = 'https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv'
df = pd.read_csv(url, dtype={'fips': str})

In [5]:
df.dtypes

date      object
county    object
state     object
fips      object
cases      int64
deaths     int64
dtype: object

In [6]:
print(df.date.min())
print(df.date.max())

2020-01-21
2020-08-13


In [7]:
unique_counties = df['county'].unique().tolist()
county_grp = df.groupby('county')

frame = {'county': unique_counties, 'state': [], 'max_cases': [], 'max_deaths': []}
for i in unique_counties:
    frame['state'].append(county_grp.get_group(i).state.iloc[0])
    frame['max_cases'].append(county_grp.get_group(i).cases.max())
    frame['max_deaths'].append(county_grp.get_group(i).deaths.max())

df1 = pd.DataFrame(frame)

print(df1.nlargest(10, 'max_cases'))
print('---------------------------------------------------------')
print(df1.nlargest(10, 'max_deaths')[['county', 'state', 'max_deaths', 'max_cases']])

            county       state  max_cases  max_deaths
25   New York City    New York     233859       23610
4      Los Angeles  California     216139        5171
152     Miami-Dade     Florida     140983        1954
3         Maricopa     Arizona     127768        2517
1             Cook    Illinois     113796        4943
47          Harris       Texas      89425        1713
53         Broward     Florida      64741         883
143         Dallas       Texas      56428         807
43           Clark      Nevada      50569         869
72       Riverside  California      44747         879
---------------------------------------------------------
            county          state  max_deaths  max_cases
25   New York City       New York       23610     233859
4      Los Angeles     California        5171     216139
1             Cook       Illinois        4943     113796
65           Wayne   Pennsylvania        2835      28715
45          Nassau       New York        2706      43795
3     

In [8]:
unique_states = df1['state'].unique().tolist()
state_grp = df1.groupby('state')

frame = {'state': unique_states, 'total_cases': [], 'total_deaths': []}
for i in unique_states:
    frame['total_cases'].append(state_grp.get_group(i).max_cases.sum())
    frame['total_deaths'].append(state_grp.get_group(i).max_deaths.sum())

df1 = pd.DataFrame(frame)

print(df1.nlargest(10, 'total_cases'))
print('---------------------------------------------')
print(df1.nlargest(10, 'total_deaths')[['state', 'total_deaths', 'total_cases']])

            state  total_cases  total_deaths
2      California       604264         11050
10        Florida       471905          7736
6           Texas       461424          8581
11       New York       365961         30284
13        Georgia       195892          4141
1        Illinois       179054          7075
3         Arizona       176425          4037
16     New Jersey       145777         12038
4   Massachusetts       145085         10573
26   Pennsylvania       134245          8806
---------------------------------------------
            state  total_deaths  total_cases
11       New York         30284       365961
16     New Jersey         12038       145777
2      California         11050       604264
4   Massachusetts         10573       145085
26   Pennsylvania          8806       134245
6           Texas          8581       461424
10        Florida          7736       471905
1        Illinois          7075       179054
33    Connecticut          4259        49155
13       

In [9]:
cols = df.columns.tolist()
print('NaN values?')
for col in cols:
    print(f'{col} column:\t{df[col].isna().values.any()}')

NaN values?
date column:	False
county column:	False
state column:	False
fips column:	True
cases column:	False
deaths column:	False


In [10]:
nan_fips_filt = df.fips.isna()
nan_df = df[nan_fips_filt]
print(nan_df)

              date         county         state fips  cases  deaths
416     2020-03-01  New York City      New York  NaN      1       0
418     2020-03-01        Unknown  Rhode Island  NaN      2       0
448     2020-03-02  New York City      New York  NaN      1       0
450     2020-03-02        Unknown  Rhode Island  NaN      2       0
482     2020-03-03  New York City      New York  NaN      2       0
...            ...            ...           ...  ...    ...     ...
430437  2020-08-13        Unknown     Tennessee  NaN    169       0
430720  2020-08-13        Unknown          Utah  NaN     16       0
430737  2020-08-13        Unknown       Vermont  NaN      8       0
430911  2020-08-13        Unknown    Washington  NaN      2       2
431034  2020-08-13        Unknown     Wisconsin  NaN   4280       7

[4189 rows x 6 columns]


In [11]:
nan_df['county'].value_counts()

Unknown          3826
New York City     166
Kansas City       147
Joplin             50
Name: county, dtype: int64

In [12]:
filt = nan_df['county'] == 'Kansas City'
nan_df[filt]

,date,county,state,fips,cases,deaths
5641,2020-03-20,Kansas City,Missouri,NaN,12,0
6631,2020-03-21,Kansas City,Missouri,NaN,13,0
7720,2020-03-22,Kansas City,Missouri,NaN,13,0
8908,2020-03-23,Kansas City,Missouri,NaN,18,0
10209,2020-03-24,Kansas City,Missouri,NaN,30,0
...,...,...,...,...,...,...
416494,2020-08-09,Kansas City,Missouri,NaN,6648,76
419715,2020-08-10,Kansas City,Missouri,NaN,6786,78
422937,2020-08-11,Kansas City,Missouri,NaN,6885,79
426159,2020-08-12,Kansas City,Missouri,NaN,7077,80


In [13]:
county_filt = df['county'] == 'New York City'

In [14]:
# Cleaning the data by making the date column a datetime dtype, dropping all rows with NaN fips value,
# and filling all NaN values with 0
df['date'] = pd.to_datetime(df['date'])
print(df.date.dtype)
#df.dropna(subset=['fips'], inplace=True)
#df.fillna(0, inplace=True)

datetime64[ns]


In [15]:
unknown_filt = nan_df.county == 'Unknown'
idxs = nan_df[unknown_filt].index.tolist()
df.drop(df.index[idxs], inplace=True)

unknown_filt = df.county == 'Unknown'
print(df[unknown_filt])

Empty DataFrame
Columns: [date, county, state, fips, cases, deaths]
Index: []


In [16]:
bad_counties_dict = {'New York City': {'county': 'New York', 'state': 'New York'}, \
                   'Kansas City': {'county': 'Jackson', 'state': 'Missouri'}, \
                   'Joplin': {'county': 'Jasper', 'state': 'Missouri'}}

nan_filt = df.fips.isnull()
for val in bad_counties_dict.items():
    city = val[0]
    county = val[1].get('county')
    state = val[1].get('state')
    
    if city == 'New York City':
        city_filt = (cities_df.city == county) & (cities_df.state_name == state)
    else:
        city_filt = (cities_df.city == city) & (cities_df.state_name == state)
    county_filt = (df.county == county) & (df.state == county)
    
    df.county.replace(city, county, inplace=True)
    fips = cities_df[city_filt]['county_fips'].sum()
    df.loc[df.county == county, 'fips'] = fips
    
print(df[nan_filt])

             date    county     state   fips   cases  deaths
416    2020-03-01  New York  New York  36061       1       0
448    2020-03-02  New York  New York  36061       1       0
482    2020-03-03  New York  New York  36061       2       0
518    2020-03-04  New York  New York  36061       2       0
565    2020-03-05  New York  New York  36061       4       0
...           ...       ...       ...    ...     ...     ...
426159 2020-08-12   Jackson  Missouri  29095    7077      80
426474 2020-08-12  New York  New York  36061  233422   23602
429382 2020-08-13    Jasper  Missouri  29097     549      21
429383 2020-08-13   Jackson  Missouri  29095    7285      82
429699 2020-08-13  New York  New York  36061  233859   23610

[363 rows x 6 columns]


In [17]:
# Creating a column that displays the "Month, Year" to be used for the slider, and a column that contains 
# "County, State" info so we can easily get county data based off of what city is selected
df['month_year'] = df['date'].dt.strftime('%b, %Y')
df['county_state'] = df['county'] + ', ' + df['state']

cities_df['city_state_abbr'] = cities_df['city'] + ', ' + cities_df['state_id']
cities_df['county_state'] = cities_df['county_name'] + ', ' + cities_df['state_name']

print(df.dtypes)

date            datetime64[ns]
county                  object
state                   object
fips                    object
cases                    int64
deaths                   int64
month_year              object
county_state            object
dtype: object


In [18]:
df['month_year'] = df['date'].dt.strftime('%b, %Y')
df['county_state'] = df['county'] + ', ' + df['state']
df['cases_range'] = [x / 1_000 for x in df['cases']]

print(df[['month_year', 'county_state', 'cases_range']].head(), '\n')
print(df.dtypes)

  month_year           county_state  cases_range
0  Jan, 2020  Snohomish, Washington        0.001
1  Jan, 2020  Snohomish, Washington        0.001
2  Jan, 2020  Snohomish, Washington        0.001
3  Jan, 2020         Cook, Illinois        0.001
4  Jan, 2020  Snohomish, Washington        0.001 

date            datetime64[ns]
county                  object
state                   object
fips                    object
cases                    int64
deaths                   int64
month_year              object
county_state            object
cases_range            float64
dtype: object


In [19]:
cities_df['city_state_abbr'] = cities_df['city'] + ', ' + cities_df['state_id']
cities_df['county_state'] = cities_df['county_name'] + ', ' + cities_df['state_name']

print(cities_df[['city_state_abbr', 'county_state']].head(), '\n')
print(cities_df.dtypes)

    city_state_abbr           county_state
0   South Creek, WA     Pierce, Washington
1        Roslyn, WA   Kittitas, Washington
2       Sprague, WA    Lincoln, Washington
3    Gig Harbor, WA     Pierce, Washington
4  Lake Cassidy, WA  Snohomish, Washington 

city               object
state_id           object
state_name         object
county_fips         int64
county_name        object
city_state_abbr    object
county_state       object
dtype: object


In [20]:
# Create a dict where the keys start at 1 and increases by one for each month, and the values are the 
# associated "month, Year". This helps with converting the numerical slider value to an actual "month_ Year" format
month_year_dict = {i+1: month_year for i, month_year in enumerate(df.month_year.unique().tolist())}
print(month_year_dict)

{1: 'Jan, 2020', 2: 'Feb, 2020', 3: 'Mar, 2020', 4: 'Apr, 2020', 5: 'May, 2020', 6: 'Jun, 2020', 7: 'Jul, 2020', 8: 'Aug, 2020'}


In [21]:
app = dash.Dash(__name__)
#app = JupyterDash(__name__)

In [22]:
# Create the layout for the dash app
app.layout = html.Div([
    # Choropleth map with slider
    html.Div([
        html.H1('Cases of Covid-19 in the U.S.', style={'text-align': 'center', 'font-family': 'Roboto Bold', 'letter-spacing': '2px'}),
        dcc.Graph(id='choropleth-with-slider'),
        dcc.Slider(
            id='month-year-slider',
            min=1,
            max=len(month_year_dict),
            value=len(month_year_dict),
            marks=month_year_dict,
            step=None)],
        style={'display': 'inline-block', 'width': '70%', 'padding-left': '20px', 'font-family': 'Roboto'}
    ),
    
    # County data header and drop down
    html.Div([
        html.H1('County Data', style={'text-align': 'center', 'font-family': 'Roboto Bold', 'letter-spacing': '2px'}),
        # Cities drop down menu
        html.Div([
            dcc.Dropdown(
                id='city-selector',
                options=[{'label': i, 'value': i} for i in cities_df.city_state_abbr],
                placeholder='Select a city')],
            style={'font-family': 'Roboto'}
        ),
        
        # County data and stats
        html.Br(),
        html.Div([
            
            # Displays selected "City, State"
            html.Div([
                html.P('City, State', id='city-state-name', style={'color': '#c2c2c2'})],
                style={'line-height': '15%', 'font-family': 'Roboto Light', 'font-weight': '800'}
            ),
            
            # Displays "County" based off selected city
            html.Div([
                html.H2('County', id='county-name', style={'color': '#c2c2c2'})],
                style={'line-height': '7%', 'padding-top': '.04px', 'font-family': 'Roboto Medium'}
            ),
            
            # Displays the most recent date that data was aquired
            html.Div([
                html.P('As of', id='most-recent-date', style={'color': '#c2c2c2'})],
                style={'font-size': '70%', 'line-height': '70%', 'font-family': 'Roboto Light', 'letter-spacing': '1px'}
            ),
            
            # Displays cases amd deaths county data
            html.Div([
                
                # Displays headers above cases and deaths count
                html.Div([
                    html.Small('TOTAL CASES', id='county-cases-label', style={'color': '#c2c2c2'}),
                    html.Small('TOTAL DEATHS', id='county-deaths-label', style={'color': '#c2c2c2'})
                ],
                # style for text "cases" and total cases
                style={'display': 'grid', 'grid-template-columns': '1fr 1fr', 'grid-gap': '70px', \
                       'font-family': 'Roboto Light', 'font-size': '85%', 'padding-top': '10px'}
                ),
                
                # Displays the total cases and deaths for the county
                html.Div([
                    # Total cases
                    html.Div([
                        html.H2('0', id='county-cases', style={'color': '#c2c2c2'})]),
                    # Total deaths
                    html.Div([
                        html.H2('0', id='county-deaths', style={'color': '#c2c2c2'})])
                ],
                style={'line-height': '20%', 'display': 'grid', 'grid-template-columns': '1fr 1fr', \
                       'grid-gap': '70px', 'letter-spacing': '3px', 'font-family': 'Roboto Medium', 'font-weight': '2500'}
                ),
                
                # Displays the increased or decreased percentage
                html.Div([
                    html.Small('% increase', id='cases-pct', style={'color': '#c2c2c2'}),
                    html.Small('% increase', id='deaths-pct', style={'color': '#c2c2c2'})
                ],
                style={'display': 'grid', 'grid-template-columns': '1fr 1fr', 'grid-gap': '70px', \
                       'line-height': '25%', 'padding-bottom': '20px', 'font-family': 'Roboto', \
                       'font-size': '80%', 'letter-spacing': '1px'}
                ),
                
                # Display a date to reference for an increase or decrease
                html.Div([
                    html.Small('since', id='cases-pct-date', style={'color': '#c2c2c2'}),
                    html.Small('since', id='deaths-pct-date', style={'color': '#c2c2c2'})
                ],
                style={'display': 'grid', 'grid-template-columns': '1fr 1fr', 'grid-gap': '70px', \
                       'line-height': '25%', 'padding-bottom': '20px', 'font-family': 'Roboto', \
                       'font-size': '80%', 'letter-spacing': '1px', 'margin-top': '-8px'}
                )
            ],
            style={'justify-content': 'center', 'width': '100%', 'text-align': 'center'}
            ),
            
            # Thematic break
            html.Hr(style={'border-top': '1px solid'}),
            
            # Displays the state of the selected city
            html.Div([
                html.H4('State', id='state-name', style={'color': '#c2c2c2'})
            ],
            style={'font-family': 'Roboto Medium'}
            ),
            
            # State data
            html.Div([
                
                # Display headers for state cases and deaths
                html.Div([
                    html.Small('TOTAL CASES', id='state-cases-label', style={'color': '#c2c2c2', 'font-size': '70%'}),
                    html.Small('TOTAL DEATHS', id='state-deaths-label', style={'color': '#c2c2c2', 'font-size': '70%'})
                ],
               style={'display': 'grid', 'grid-template-columns': '1fr 1fr', 'grid-gap': '70px', \
                       'font-family': 'Roboto Light', 'font-size': '85%', 'padding-top': '10px'}
                ),
                
                # Display the total cases and deaths for the state
                html.Div([
                    # Total cases
                    html.Div([
                        html.H3('0', id='state-cases', style={'color': '#c2c2c2'})]),
                    # Total deaths
                    html.Div([
                        html.H3('0', id='state-deaths', style={'color': '#c2c2c2'})])
                ],
                style={'line-height': '20%', 'display': 'grid', 'grid-template-columns': '1fr 1fr', \
                       'grid-gap': '70px', 'letter-spacing': '3px', 'font-family': 'Roboto Medium', 'font-weight': '100'}
                )
            ],
            style={'justify-content': 'center', 'width': '100%', 'text-align': 'center'}
            ),
            
            # Inserting thematic break followed by a barchart of monthly data on the specified county
            html.Div([
                html.Hr(style={'border-top': '1px solid'}),
                dcc.Graph(id='bar-chart')
            ],
            style={'height': '60%'}
            )
        ],
        # border for data section
        style={'border': '1px solid black', 'border-radius': '5px', 'box-shadow': '3px 3px 5px #888888', 'padding-left': '20px', 'padding-right': '20px'}
        )
    ],
    # style for right side html.Div
    style={'float': 'right', 'display': 'inline-block', 'width': '25%', 'padding-right': '20px'}
    ),
    
], 
# stlye for entire app
style={'autosizable': 'True', 'height': '100%', 'padding-bottom': '20px'}
)

In [23]:
# Creates and updates the choropleth map with a slider
@app.callback(
    Output('choropleth-with-slider', 'figure'),
    [Input('month-year-slider', 'value')])

def update_choropleth(set_month_year):
    filtered_df = df[df.month_year == month_year_dict.get(set_month_year)]
    
    fig = px.choropleth(filtered_df,
        geojson=counties,
        locations='fips', 
        color='cases_range',
        color_continuous_scale='reds',
        scope='usa',
        hover_name='county_state',
        hover_data={'cases': True, 'deaths': True, 'fips': False, 
                    'month_year': False,'county': False, 'cases_range': False},
        range_color=(0, 11),
        height=700,
        labels={'cases_range':'cases per thousand'},
        template='plotly')
    
    fig.update_layout(transition_duration=500)
    
    return fig

In [24]:
# Populating the county cases, county deaths, the percentage increased since last week for cases
# and deaths, as well as changing the style for all of those elements so they aren't greyed out.
@app.callback(
    [Output('county-name', 'children'),
     Output('county-name', 'style'),
     Output('city-state-name', 'children'),
     Output('city-state-name', 'style'),
     Output('county-cases', 'children'),
     Output('county-cases', 'style'),
     Output('county-cases-label', 'style'),
     Output('county-deaths', 'children'),
     Output('county-deaths', 'style'),
     Output('county-deaths-label', 'style'),
     Output('cases-pct', 'children'),
     Output('cases-pct', 'style'),
     Output('deaths-pct', 'children'),
     Output('deaths-pct', 'style'),
     Output('cases-pct-date', 'children'),
     Output('cases-pct-date', 'style'),
     Output('deaths-pct-date', 'children'),
     Output('deaths-pct-date', 'style'),
     Output('most-recent-date', 'children'),
     Output('most-recent-date', 'style')],
    [Input('city-selector', 'value'),
     Input('month-year-slider', 'value')])

def get_county_data(selected_city, set_month_year):
    if selected_city != None:
        city_filt = cities_df['city_state_abbr'] == selected_city
        county = cities_df[city_filt].iloc[0]['county_name']
        county_state = cities_df[city_filt].iloc[0]['county_state']
        city_group = df.groupby('county_state')

        if set_month_year != len(month_year_dict):
            current_data = False
        else:
            current_data = True

        try:
            city_df = city_group.get_group(county_state)
            month_year_filt = city_df['month_year'] == month_year_dict.get(set_month_year)
            city_month_df = city_df[month_year_filt]

            city_month_df['new_cases'] = [(val - city_month_df.iloc[i-1]['cases']) if i != 0 else val for i, val in enumerate(city_month_df['cases'])]
            city_month_df['new_deaths'] = [(val - city_month_df.iloc[i-1]['deaths']) if i != 0 else val for i, val in enumerate(city_month_df['deaths'])]

            if len(city_month_df) == 0:
                month_year_filt = df['month_year'] == month_year_dict.get(set_month_year)
                max_date = df[month_year_filt]['date'].max()
            else:
                max_date = city_month_df.date.max()
            
            if current_data:
                compare_date = max_date - timedelta(days=7)
                compare_filt = city_month_df['date'] <= compare_date
                old_cases = city_month_df[compare_filt]['new_cases'].sum()
                old_deaths = city_month_df[compare_filt]['new_deaths'].sum()
            else:
                compare_date = df.date.max()
                old_cases = city_df['cases'].max()
                old_deaths = city_df['deaths'].max()

            current_filt = city_month_df['date'] <= max_date
            new_cases = city_month_df[current_filt]['new_cases'].sum()
            new_deaths = city_month_df[current_filt]['new_deaths'].sum()

            if current_data:
                cases_change = new_cases - old_cases
                deaths_change = new_deaths - old_deaths
            else:
                cases_change = old_cases - new_cases
                deaths_change = old_deaths - new_deaths

            cases_pct = round(((cases_change / old_cases) * 100), 2)
            deaths_pct = round(((deaths_change / old_deaths) * 100), 2)
            
            if len(city_month_df) == 0:
                max_cases = 0
                max_deaths = 0
            else:
                month_number = datetime.strptime(month_year_dict.get(set_month_year), "%b, %Y").month
                my_filt = city_month_df.date.dt.month <= month_number
                
                max_cases = city_month_df[my_filt].cases.max()
                max_deaths = city_month_df[my_filt].deaths.max()
            
            filt_max_date = max_date.strftime('%A, %b %d, %Y')

            key_error = False
    
        except KeyError:
            month_year_filt = df['month_year'] == month_year_dict.get(set_month_year)
            filt_max_date = (df[month_year_filt]['date'].max()).strftime('%A, %b %d, %Y')
            key_error = True

        most_recent_date = df.date.max().strftime('%b %d, %Y')
        name_style = {'color': '#1f1f1f', 'letter-spacing': '2px'}
        cases_style = {'color': '#d91e1e'}
        deaths_style = {'color': '#1f1f1f'}
        label_style = {'color': '#1f1f1f'}

        if key_error:
            if current_data:
                pct = '0% increase'
                pct_date = 'since last week'
            else:
                pct = '0% decrease'
                pct_date = f'since {most_recent_date}'
            return county, name_style, selected_city, name_style, 0, cases_style, label_style, 0, deaths_style, label_style, pct, label_style, pct, label_style, pct_date, label_style, pct_date, label_style, f'As of {filt_max_date}', label_style
        else:
            if current_data:
                return  county, name_style, selected_city, name_style, ('{:,}'.format(max_cases)), cases_style, label_style, ('{:,}'.format(max_deaths)), deaths_style, label_style, \
                        f'{cases_pct}% increase', label_style, f'{deaths_pct}% increase', label_style, \
                        'since last week', label_style, 'since last week', label_style, f'As of {filt_max_date}', label_style
            else:
                 return county, name_style, selected_city, name_style, ('{:,}'.format(max_cases)), cases_style, label_style, ('{:,}'.format(max_deaths)), deaths_style, label_style, \
                        f'{cases_pct}% decrease', label_style, f'{deaths_pct}% decrease', label_style, f'since {most_recent_date}', \
                        label_style, f'since {most_recent_date}', label_style, f'As of {filt_max_date}', label_style
        
    else:
        style = {'color': '#c2c2c2'}
        pct = '% increase'
        pct_date = 'since'
        return 'County', style, 'City, State', style, 0, style, style, 0, style, style, pct, style, pct, style, pct_date, style, pct_date, style, 'As of', style

In [25]:
# Populating the state name, state cases, and state deaths. As well as changing
# the style of these elements so they aren't greyed out.
@app.callback(
    [Output('state-name', 'children'),
     Output('state-name', 'style'),
     Output('state-cases', 'children'),
     Output('state-cases', 'style'),
     Output('state-cases-label', 'style'),
     Output('state-deaths', 'children'),
     Output('state-deaths', 'style'),
     Output('state-deaths-label', 'style')],
    [Input('city-selector', 'value'),
     Input('month-year-slider', 'value')])

def get_state_data(selected_city, set_month_year):
    if selected_city != None:
        city_grp = cities_df.groupby('city_state_abbr')
        state = city_grp.get_group(selected_city).iloc[0]['state_name']
        state_grp = df.groupby('state')
        state_df = state_grp.get_group(state)
        month_year_filt = state_df['month_year'] == month_year_dict.get(set_month_year)
        length = len(state_df[month_year_filt])

        state_month_df = state_df[month_year_filt]
        unique_counties = state_month_df.county.unique()

        state_cases = 0
        state_deaths = 0
        for county in unique_counties:
            county_filt = state_month_df['county'] == county
            state_cases += state_month_df[county_filt]['cases'].max()
            state_deaths += state_month_df[county_filt]['deaths'].max()
        
        state_style = {'color': '#1f1f1f', 'letter-spacing': '2px'}
        cases_style = {'color': '#d91e1e'}
        deaths_style = {'color': '#1f1f1f'}
        label_style = {'color': '#1f1f1f', 'font-size': '70%'}
        return state, state_style, ('{:,}'.format(state_cases)), cases_style, label_style, ('{:,}'.format(state_deaths)), deaths_style, label_style
    
    else:
        style1 = {'color': '#c2c2c2'}
        style2 = {'color': '#c2c2c2', 'font-size': '70%'}
        return 'State', style1, 0, style1, style2, 0, style1, style2

In [26]:
# Creating the barchart that will display the number of cases and deaths in each month for the selected county.
@app.callback(
    Output('bar-chart', 'figure'),
    [Input('city-selector', 'value')])

def update_bar(selected_city):
    dummy_city = False
    if selected_city == None:
        rand = randrange(len(cities_df))
        dummy_city = True
        selected_city = cities_df.iloc[rand]['city_state_abbr']
    
    city_filt = cities_df['city_state_abbr'] == selected_city
    county = cities_df[city_filt].iloc[0]['county_name']
    county_state = cities_df[city_filt].iloc[0]['county_state']

    try:
        county_group = df.groupby('county_state')
        county_df = county_group.get_group(county_state)
        county_df['month_year'] = [i.strftime('%b, %Y') for i in county_df['date']]
        county_df['new_cases'] = [(val - county_df.iloc[i-1]['cases']) if i != 0 else val for i, val in enumerate(county_df['cases'])]
        county_df['new_deaths'] = [(val - county_df.iloc[i-1]['deaths']) if i != 0 else val for i, val in enumerate(county_df['deaths'])]

        months = county_df.month_year.unique().tolist()
        m_grp = county_df.groupby('month_year')
        cases = []
        deaths = []
        for i in months:
            m_cases = (m_grp.get_group(i))['new_cases'].sum()
            m_deaths = (m_grp.get_group(i))['new_deaths'].sum()
            cases.append(m_cases)
            deaths.append(m_deaths)

        frame = {'month_year': months, 'cases': cases, 'deaths': deaths}
    except KeyError:
        months = df.month_year.unique().tolist()
        cases = []
        deaths = []
        for i in months:
            cases.append(0)
            deaths.append(0)
        frame = {'month_year': months, 'cases': cases, 'deaths': deaths}
    
    barchart_df = pd.DataFrame(frame)
    
    if dummy_city:
        cases_color = '#c2c2c2'
        deaths_color = '#c2c2c2'
        font = {'color': '#c2c2c2'}
        title = 'County'
    else:
        cases_color = '#d91e1e'
        deaths_color = '#1f1f1f'
        font = {'color': '#1f1f1f'}
        title = f'{county} County'

    fig = go.Figure()
    fig.add_trace(go.Bar(x=barchart_df.month_year,
                         y=barchart_df.cases,
                         text=barchart_df.cases,
                         name='New cases',
                         marker_color=cases_color,
                         hoverinfo='name+y')),
    fig.add_trace(go.Bar(x=barchart_df.month_year,
                         y=barchart_df.deaths,
                         text=barchart_df.deaths,
                         name='New deaths',
                         marker_color=deaths_color,
                         hoverinfo='name+y'))
    fig.update_layout(
        margin=dict(l=20, r=20, t=20, b=20),
        title={
            'text': title,
            'y':1,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        height=400,
        font = font,
        xaxis = dict(
            tickangle=30,
            tickfont_size=12),
        yaxis=dict(
            tickfont_size=12,),
        legend=dict(
            orientation='h',
            xanchor='center',
            x=0.5,
            yanchor='bottom',
            y=1,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'),
        barmode='group',
        bargap=0.2,
        bargroupgap=0.05)

    return fig

In [ ]:
# app.run_server(mode='jupyterlab')
app.run_server()